# Preprocessing: SLCH002


This markdown files loads the clean data and does a bipolar rereference as well as epochs the data


In [1]:
import matplotlib
# matplotlib.use("Qt5Agg")
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import signal, stats
import mat73
import re
from neurodsp.timefrequency import compute_wavelet_transform
from BCI2kReader import BCI2kReader as b2k
import os
import mne
from tabulate import tabulate
import IPython
import seaborn as sns

In [2]:
#%matplotlib qt5

In [3]:
## Prep paths ##

subject = 'SLCH002'
raw_data_dir = f"/home/brooke/pacman/raw_data/{subject}"
preproc_data_dir = f"/home/brooke/pacman/preprocessing/{subject}/ieeg"

In [5]:
## Load Data ##

# load filtered data #
filtered_clean_fif = mne.io.Raw(f"{raw_data_dir}/ieeg/{subject}_notched_filtered_clean_ieeg.fif")

# load raw data #
raw_clean_fif = mne.io.Raw(f"{raw_data_dir}/ieeg/{subject}_raw_clean_ieeg.fif")



Opening raw data file /home/brooke/pacman/raw_data/SLCH002/ieeg/SLCH002_notched_filtered_clean_ieeg.fif...
    Range : 0 ... 2373999 =      0.000 ...  1186.999 secs
Ready.
Opening raw data file /home/brooke/pacman/raw_data/SLCH002/ieeg/SLCH002_notched_filtered_clean_ieeg-1.fif...
    Range : 2374000 ... 2846599 =   1187.000 ...  1423.300 secs
Ready.
Opening raw data file /home/brooke/pacman/raw_data/SLCH002/ieeg/SLCH002_raw_clean_ieeg.fif...
    Range : 0 ... 2373999 =      0.000 ...  1186.999 secs
Ready.
Opening raw data file /home/brooke/pacman/raw_data/SLCH002/ieeg/SLCH002_raw_clean_ieeg-1.fif...
    Range : 2374000 ... 2846599 =   1187.000 ...  1423.300 secs
Ready.


## Bipolar Rereferencing

In [6]:
# helper functions

def probe_and_num(elec_str): 
    ''' This convenience function takes an electrode string like LHH1 and outputs ('LHH', 1)
    '''
    if '_' in elec_str:
        return None, None
    else:   
        regex_str = '(\D+)(\d+)' # group of letters followed by group of digits
        matcher = re.compile(regex_str,re.IGNORECASE|re.DOTALL).search(elec_str)
        if matcher:
            probe, num = matcher.groups()
            return probe, int(num)
        else:
            return None, None
    
def find_bipolar_pair(ch, labels, remove):
    ''' Find the pair of a given electrode for bipolar referencing.
        Given a single *ch* and a list of *labels*, some of which you want to *remove*,
        this finds the next channel on that probe that isn't meant to be removed (WM is ok).
    '''
    bipolar_pair = None
    probe, num = probe_and_num(ch)
    other_in_probe = [other_ch for other_ch in labels if probe_and_num(other_ch)[0]==probe]
    for i in range(len(other_in_probe)-num):
        next_ch = probe + str(num+i+1)
        if next_ch in remove or next_ch == 'STI':
            continue
        else:
            bipolar_pair = next_ch
            break
    return bipolar_pair

In [26]:
# Here I iterate through the electrodes in my ROIs and match them up with their bipolar pair
pairs = []
anode = []
cathode = []
pairs_name = []
pairs_map = {}
remove = filtered_clean_fif.info['bads']
labels = filtered_clean_fif.info['ch_names']
for ch in labels:
    # if a channel is meant to be removed, it doesn't get to be in a bipolar pair
    if ch in remove or ch == 'STI':
        print(f"{ch} noref")
    else:
        pair = find_bipolar_pair(ch, labels, remove)
        if pair:
            anode.append(ch)
            cathode.append(pair)
            pairs.append((ch, pair))
            pairs_map[ch] = f"{ch}-{pair}"
            pairs_name.append(f"{ch}-{pair}")
            print(ch, pair)

REF1 noref
REF2 noref
A1 A2
A2 A3
A3 A4
A4 A5
A5 A6
A6 A7
A7 A8
A8 A9
A9 A10
A10 A11
A11 A12
A12 A13
A13 A14
A14 A15
A15 A16
B1 B2
B2 B3
B3 B4
B4 B5
B5 B6
B6 B7
B7 B8
B8 B9
B9 B10
B10 B11
B11 B12
B12 B13
B13 B14
B14 B15
B15 B16
C1 C2
C2 C3
C3 C4
C4 C5
C5 C6
C6 C7
C7 C8
C8 C9
C9 C10
C10 C11
C11 C12
C12 C13
C13 C14
C14 C15
C15 C16
D1 D2
D2 D3
D3 D4
D4 D7
D5 noref
D6 noref
D7 D8
D8 D9
D9 D10
E1 E2
E2 E3
E3 E4
E4 E5
E5 E6
E6 E8
E7 noref
E8 E9
E9 E10
F1 F2
F2 F3
F3 F4
F4 F5
F5 F6
F6 F7
F7 F8
F8 F9
F9 F10
G1 G2
G2 G3
G3 G4
G4 G5
G5 G6
G6 G7
G7 G8
G8 G9
G9 G10
h1 h2
h2 h3
h3 h4
h4 h5
h5 h6
h6 h7
h7 h8
h8 h9
h9 h10
i1 i2
i2 i3
i3 i6
i4 noref
i5 noref
i6 i7
i7 i8
i8 i9
i9 i10
i10 i11
i11 i12
i12 i13
i13 i14
EKG1 noref
EKG2 noref
J1 noref
J2 noref
J3 noref
J4 J5
J5 J6
J6 J7
J7 J8
J8 J9
J9 J10
J10 J11
J11 J12
J12 J13
J13 J14
K1 K2
K2 K3
K3 K4
K4 K5
K5 K6
K6 K7
K7 K8
K8 K9
K9 K10
K10 K11
K11 K12
K12 K13
K13 K14
K14 K15
K15 K16
L1 L2
L2 L3
L3 L4
L4 L5
L5 L6
L6 L7
L7 L8
L8 L9
L9 L10
L10 L11
L11 L12


In [27]:
## Apply Rereference #

if filtered_clean_fif.info['ch_names'] == raw_clean_fif.info['ch_names'] and filtered_clean_fif.info['bads'] == raw_clean_fif.info['bads']:
    
    # load filtered data
    filtered_clean_fif.load_data()
    
    # set filtered reference 
    bp_filt_fif = mne.set_bipolar_reference(filtered_clean_fif, anode = anode, cathode = cathode)
    
    # save
    bp_filt_fif.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif", overwrite = True)
    
    # clear var
    bp_filt_fif = []
    filtered_clean_fif = []
    
    # load raw data
    raw_clean_fif.load_data()
    
    # set raw reference 
    bp_raw_fif = mne.set_bipolar_reference(raw_clean_fif, anode = anode, cathode = cathode)
    
    # save
    bp_raw_fif.save(f"{preproc_data_dir}/{subject}_bp_raw_clean_data.fif", overwrite = True)

    
    # clear var
    bp_raw_fif = []
    raw_clean_fif = []    
    
    

Reading 0 ... 2846599  =      0.000 ...  1423.300 secs...
sEEG channel type selected for re-referencing
Creating RawArray with float64 data, n_channels=164, n_times=2846600
    Range : 0 ... 2846599 =      0.000 ...  1423.300 secs
Ready.
Added the following bipolar channels:
A1-A2, A2-A3, A3-A4, A4-A5, A5-A6, A6-A7, A7-A8, A8-A9, A9-A10, A10-A11, A11-A12, A12-A13, A13-A14, A14-A15, A15-A16, B1-B2, B2-B3, B3-B4, B4-B5, B5-B6, B6-B7, B7-B8, B8-B9, B9-B10, B10-B11, B11-B12, B12-B13, B13-B14, B14-B15, B15-B16, C1-C2, C2-C3, C3-C4, C4-C5, C5-C6, C6-C7, C7-C8, C8-C9, C9-C10, C10-C11, C11-C12, C12-C13, C13-C14, C14-C15, C15-C16, D1-D2, D2-D3, D3-D4, D4-D7, D7-D8, D8-D9, D9-D10, E1-E2, E2-E3, E3-E4, E4-E5, E5-E6, E6-E8, E8-E9, E9-E10, F1-F2, F2-F3, F3-F4, F4-F5, F5-F6, F6-F7, F7-F8, F8-F9, F9-F10, G1-G2, G2-G3, G3-G4, G4-G5, G5-G6, G6-G7, G7-G8, G8-G9, G9-G10, h1-h2, h2-h3, h3-h4, h4-h5, h5-h6, h6-h7, h7-h8, h8-h9, h9-h10, i1-i2, i2-i3, i3-i6, i6-i7, i7-i8, i8-i9, i9-i10, i10-i11, i11-i12, i12

/tmp/ipykernel_3841/1336304392.py:12: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/SLCH002/ieeg/SLCH002_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filt_fif.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif", overwrite = True)


Overwriting existing file.
Writing /home/brooke/pacman/preprocessing/SLCH002/ieeg/SLCH002_bp_filtered_clean_data-1.fif
Closing /home/brooke/pacman/preprocessing/SLCH002/ieeg/SLCH002_bp_filtered_clean_data-1.fif
Closing /home/brooke/pacman/preprocessing/SLCH002/ieeg/SLCH002_bp_filtered_clean_data.fif
[done]
Reading 0 ... 2846599  =      0.000 ...  1423.300 secs...
sEEG channel type selected for re-referencing
Creating RawArray with float64 data, n_channels=164, n_times=2846600
    Range : 0 ... 2846599 =      0.000 ...  1423.300 secs
Ready.
Added the following bipolar channels:
A1-A2, A2-A3, A3-A4, A4-A5, A5-A6, A6-A7, A7-A8, A8-A9, A9-A10, A10-A11, A11-A12, A12-A13, A13-A14, A14-A15, A15-A16, B1-B2, B2-B3, B3-B4, B4-B5, B5-B6, B6-B7, B7-B8, B8-B9, B9-B10, B10-B11, B11-B12, B12-B13, B13-B14, B14-B15, B15-B16, C1-C2, C2-C3, C3-C4, C4-C5, C5-C6, C6-C7, C7-C8, C8-C9, C9-C10, C10-C11, C11-C12, C12-C13, C13-C14, C14-C15, C15-C16, D1-D2, D2-D3, D3-D4, D4-D7, D7-D8, D8-D9, D9-D10, E1-E2, E2-E3

/tmp/ipykernel_3841/1336304392.py:25: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/SLCH002/ieeg/SLCH002_bp_raw_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_raw_fif.save(f"{preproc_data_dir}/{subject}_bp_raw_clean_data.fif", overwrite = True)


Overwriting existing file.
Writing /home/brooke/pacman/preprocessing/SLCH002/ieeg/SLCH002_bp_raw_clean_data-1.fif
Closing /home/brooke/pacman/preprocessing/SLCH002/ieeg/SLCH002_bp_raw_clean_data-1.fif
Closing /home/brooke/pacman/preprocessing/SLCH002/ieeg/SLCH002_bp_raw_clean_data.fif
[done]


In [ ]:
# Visualize it #

filtered_notch_bp_data.plot(events=events, color='b', bad_color = 'cyan', n_channels = 1, clipping = None, event_color = 'r')

## Epoching the data 

Longest usable trial is 8.45 seconds, round up to 9 and add 3 seconds for buffer. 1 second baseline if we want, plus 3 seconds buffer

In [5]:
## Epoching the raw data ##

# load data 
bp_raw_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_raw_clean_data.fif")
events = mne.find_events(bp_raw_fif, output='step', consecutive = False, stim_channel='STI')

# create events
    
# only epoch good channels
channels_of_interest =  [x for x in bp_raw_fif.info['ch_names'] if x not in bp_raw_fif.info['bads']]

# epoch the data
epoched_data = mne.Epochs(bp_raw_fif, events, 
                          event_id = 1, tmin = -4, tmax = 12, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# save the data
epoched_data.save(f"{preproc_data_dir}/{subject}_bp_clean_pres-locked_ieeg.fif", overwrite = True, split_size = '1.9GB')


Opening raw data file /home/brooke/pacman/preprocessing/SLCH002/ieeg/SLCH002_bp_raw_clean_data.fif...
    Range : 0 ... 2529999 =      0.000 ...  1264.999 secs
Ready.
Opening raw data file /home/brooke/pacman/preprocessing/SLCH002/ieeg/SLCH002_bp_raw_clean_data-1.fif...
    Range : 2530000 ... 2846599 =   1265.000 ...  1423.300 secs
Ready.


/tmp/ipykernel_33248/4245955747.py:4: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/SLCH002/ieeg/SLCH002_bp_raw_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_raw_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_raw_clean_data.fif")


480 events found
Event IDs: [0 1]
Not setting metadata
240 matching events found
No baseline correction applied
0 projection items activated
Loading data for 240 events and 32001 original time points ...


/tmp/ipykernel_33248/4245955747.py:18: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/SLCH002/ieeg/SLCH002_bp_clean_pres-locked_ieeg.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epoched_data.save(f"{preproc_data_dir}/{subject}_bp_clean_pres-locked_ieeg.fif", overwrite = True, split_size = '1.9GB')


0 bad epochs dropped
Loading data for 1 events and 32001 original time points ...
Splitting into 3 parts
Loading data for 80 events and 32001 original time points ...
Loading data for 80 events and 32001 original time points ...
Loading data for 80 events and 32001 original time points ...


## Trial End

In [4]:
## Epoching the raw data ##

# load data 
bp_raw_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_raw_clean_data.fif")
    
# only epoch good channels
channels_of_interest =  [x for x in bp_raw_fif.info['ch_names'] if x not in bp_raw_fif.info['bads']]

# epoch the data
epoched_data = mne.Epochs(bp_raw_fif, events, 
                          event_id = 0, tmin = -5, tmax = 5, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# save the data
epoched_data.save(f"{preproc_data_dir}/{subject}_bp_clean_end-locked_ieeg.fif", overwrite = True, split_size = '1.9GB')

Opening raw data file /home/brooke/pacman/preprocessing/SLCH002/ieeg/SLCH002_bp_raw_clean_data.fif...
    Range : 0 ... 2529999 =      0.000 ...  1264.999 secs
Ready.
Opening raw data file /home/brooke/pacman/preprocessing/SLCH002/ieeg/SLCH002_bp_raw_clean_data-1.fif...
    Range : 2530000 ... 2846599 =   1265.000 ...  1423.300 secs
Ready.


/tmp/ipykernel_11837/4119140940.py:4: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/SLCH002/ieeg/SLCH002_bp_raw_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_raw_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_raw_clean_data.fif")


480 events found
Event IDs: [0 1]
Not setting metadata
240 matching events found
No baseline correction applied
0 projection items activated
Loading data for 240 events and 20001 original time points ...


/tmp/ipykernel_11837/4119140940.py:18: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/SLCH002/ieeg/SLCH002_bp_clean_end-locked_ieeg.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epoched_data.save(f"{preproc_data_dir}/{subject}_bp_clean_end-locked_ieeg.fif", overwrite = True, split_size = '1.9GB')


1 bad epochs dropped
Loading data for 1 events and 20001 original time points ...
Splitting into 2 parts
Loading data for 120 events and 20001 original time points ...
Loading data for 119 events and 20001 original time points ...


Okay, what we need to do is pul the sample of the last dot eaten on each trial

and then also the last turn around

looking for a comparison between these moments on ghost and no ghost trials

we can either get the average and then compare peaks via foof groups in hc and ofc and anterior cingulate

## Last Away

In [6]:
## Load Behavioral Data ##
last_away_data = pd.read_csv(f"{raw_data_dir}/behave/{subject}_last_away_events.csv")
last_away_data

,neural_trial_numeric,TrialType,sample,sample_before,event
0,0,8,39400,0,1
1,1,12,59400,0,1
2,2,3,71000,0,1
3,4,16,110400,0,1
4,5,2,126000,0,1
...,...,...,...,...,...
220,234,10,2740200,0,1
221,235,19,2750000,0,1
222,236,2,2761200,0,1
223,237,18,2772600,0,1


In [7]:
# create events
last_away_events = last_away_data[['sample', 'sample_before', 'event']].copy().to_numpy()


In [8]:
# load data 
bp_raw_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_raw_clean_data.fif")

# only epoch good channels
channels_of_interest =  [x for x in bp_raw_fif.info['ch_names'] if x not in bp_raw_fif.info['bads']]


# epoch the data
last_away_epochs = mne.Epochs(bp_raw_fif, last_away_events, 
                          event_id = 1, tmin = -5, tmax =5, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# # save the data
last_away_epochs.save(f"{preproc_data_dir}/{subject}_bp_raw_clean_last_away_events.fif", overwrite = True, split_size = '1.9GB')


Opening raw data file /home/brooke/pacman/preprocessing/SLCH002/ieeg/SLCH002_bp_raw_clean_data.fif...
    Range : 0 ... 2529999 =      0.000 ...  1264.999 secs
Ready.
Opening raw data file /home/brooke/pacman/preprocessing/SLCH002/ieeg/SLCH002_bp_raw_clean_data-1.fif...
    Range : 2530000 ... 2846599 =   1265.000 ...  1423.300 secs
Ready.
Not setting metadata
225 matching events found
No baseline correction applied
0 projection items activated
Overwriting existing file.
Loading data for 225 events and 20001 original time points ...


/tmp/ipykernel_2575/2975798972.py:2: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/SLCH002/ieeg/SLCH002_bp_raw_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_raw_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_raw_clean_data.fif")
/tmp/ipykernel_2575/2975798972.py:14: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/SLCH002/ieeg/SLCH002_bp_raw_clean_last_away_events.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  last_away_epochs.save(f"{preproc_data_dir}/{subject}_bp_raw_clean_last_away_events.fif", overwrite = True, split_size = '1.9GB')


0 bad epochs dropped
Loading data for 1 events and 20001 original time points ...
Splitting into 2 parts
Loading data for 113 events and 20001 original time points ...
Loading data for 112 events and 20001 original time points ...


## Last Dot

In [4]:
## Load Behavioral Data ##
last_dot_data = pd.read_csv(f"{raw_data_dir}/behave/{subject}_last_dot_events.csv")
last_dot_data

,trial_numeric,TrialType,sample,sample_before,event
0,2,8,96500,0,1
1,3,11,119100,0,1
2,5,10,137800,0,1
3,6,15,157400,0,1
4,8,20,194700,0,1
...,...,...,...,...,...
397,446,17,5770200,0,1
398,447,15,5782200,0,1
399,448,18,5793400,0,1
400,449,19,5811600,0,1


In [6]:
# filter to good epochs and fix trial indexing
last_dot_data['trial_numeric'] = last_dot_data['trial_numeric'] - 1

# create events
last_dot_events = last_dot_data[['sample', 'sample_before', 'event']].copy().to_numpy()


In [9]:
# epoch the data
last_dot_epochs = mne.Epochs(bp_raw_fif, last_dot_events, 
                          event_id = 1, tmin = -.75 * 4, tmax =.75 * 3, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# # save the data
last_dot_epochs.save(f"{preproc_data_dir}/{subject}_bp_raw_clean_last_dot_events.fif", overwrite = True, split_size = '1.9GB')


Not setting metadata
402 matching events found
No baseline correction applied
0 projection items activated
Overwriting existing file.
Loading data for 402 events and 10501 original time points ...


/tmp/ipykernel_2043602/260881574.py:7: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH016/ieeg/BJH016_bp_raw_clean_last_dot_events.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  last_dot_epochs.save(f"{preproc_data_dir}/{subject}_bp_raw_clean_last_dot_events.fif", overwrite = True, split_size = '1.9GB')


0 bad epochs dropped
Loading data for 1 events and 10501 original time points ...
Splitting into 2 parts
Loading data for 201 events and 10501 original time points ...
Loading data for 201 events and 10501 original time points ...
